# Подключение **DeepPavlov** чатбота к **Яндекс.Алиса**

В этом туториале я расскажу как разработать и подключить чатбота на основе библиотеки **DeepPavlov** к **Яндекс.Алиса**. Наш бот сможет приветствовать, прощаться и отвечать на вопросы (на основе списка FAQ - часто задаваемых вопросов). Более детальное руководство по созданию autoFAQ ботов на основе библиотеки DeepPavlov вы сможете найти в статье [Simple intent recognition and question answering with DeepPavlov](https://medium.com/deeppavlov).

## Установка

Для начала установите Python 3.6 и активируйте виртуальную среду разработки (`source activate py36`). Затем скачайте библиотеку DeepPavlov.

In [ ]:
!pip install -q deeppavlov

## Разработка

Создайте скиллы для приветствия, прощания, и скилл на случай если запрос пользователя не удовлетворяет ни одной из категорий. Параметр responses определяет варианты ответа чатбота на запросы из параметра patters. Если при определении скилла параметр `pattens` не задан, то этот скилл становится скиллом "заглушкой", то есть он вызывается в том случае если ни один из скиллов не сработал. Параметр `default_confidence` задает минимальный порог уверенности при котором скилл-заглушка сработает.

In [ ]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill

hello = PatternMatchingSkill(responses=['Привет!', 'Здравствуйте', 'Добрый день'], patterns=['Привет', 'Здравствуйте', 'Добрый день'])
bye = PatternMatchingSkill(responses=['Пока!', 'До свидания, надеюсь смог вам помочь', 'До встречи!'], patterns=['До свидания', 'Пока', 'Спасибо за помощь'])
fallback = PatternMatchingSkill(responses=['Я не понял, но могу попробовать ответить на другой вопрос', 'Я не понял, задайте другой вопрос'], default_confidence = 0.5)

Создайте объект класса `SimilarityMatchingSkill`, который отвечает на запрос пользователя на основе списка часто задаваемых вопросов. Объект имеет следующие параметры
* `data_path` - путь к csv файлу с данными
* `x_col_name` - имя колонки с вопросами в csv файле (Question, по умолчанию)
* `y_col_name` - имя колонки с ответами в csv файле (Answer, по умолчанию)
* `edit_dict` - `dict` с параметрами конфигурации
* `save_load_path` - путь куда сохранить натренированную модель
* `train` - тренировать ли модель?

In [ ]:
from deeppavlov.contrib.skills.similarity_matching_skill import SimilarityMatchingSkill

faq = SimilarityMatchingSkill(data_path = 'http://files.deeppavlov.ai/faq/dataset.csv',
                              x_col_name = 'Question', 
                              y_col_name = 'Answer',
                              save_load_path = './model',
                              config_type = 'tfidf_autofaq',
                              train = True)

`SimilarityMatchingSkill` это скилл который позволяет использовать модели классификации интентов пользователя. Подробнее о этих моделях вы сможете узнать из нашего [блога](https://medium.com/deeppavlov).
Натренировав модель и сохранив в `save_load_path`, вы можете использовать ее указав путь загрузки `faq = SimilarityMatchingSkill(load_path='./model')`.

Последний шаг объединить все скиллы в агент, и настроить параметр выбора скилла. Параметр `HighestConfidenceSelector` определяет, что будет вызван скилл с наивысшей уверенностью (confidence).

In [ ]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

agent = DefaultAgent([hello, bye, faq, fallback], skills_selector=HighestConfidenceSelector())

Убедитесь, что агент корректо отвечает на запросы.

In [ ]:
print(agent(['Привет', 'Мне нужен личный кабинет', 'Пока']))

Далее запустите сервер с указанием пути для запросов `endpoint='faq'` и порта подключения `port=5000`

In [ ]:
from deeppavlov.utils.alice import start_agent_server

start_agent_server(agent, host='0.0.0.0', port=5000, endpoint='/faq')

Обратите внимание, что Яндекс.Диалоги в качестве **Webhook URL** требует указывать сервер с внешним IP адресом и доступом по протоколу *https*. 
Для быстрого прототипирования вы можете использовать [ngrok](https://ngrok.com/). **Ngrok** позволит вам создавать туннель для доступа к вашему серверу с **DeepPavlov** в локальной сети, для этого запустите *ngrok http 5000* на вашем сервере с DeepPavlov. В ответ на это будет создано два туннеля, по одному на протоколы http и https.
Скопируйте адрес туннеля для https, добавьте к линку эндпоинт */faq*, итоговый линк будет **Webhook URL** для нашего Яндекс.Диалога.

# Ссылки

[DeepPavlov repository](https://github.com/deepmipt/DeepPavlov)

[DeepPavlov demo page](https://demo.deeppavlov.ai)

[DeepPavlov documentation](https://docs.deeppavlov.ai)

[DeepPavlov blog](https://medium.com/deeppavlov)